In [1]:
from bonsai_graph.manager import *
from bonsai_graph.ops import *
from bonsai_graph.helpers import show_time
import time
import numpy as np
import random

%load_ext autoreload
%autoreload 2

Using backend: pytorch


In [2]:
kwargs = {
    'dataset':'pubmed',
    'head_num':2,
    'num_layers':2,
    'prune':True,
    'lr':.005,
    'epochs':64,
    'supervised':False,
}

manager = CitationGNNManager(kwargs)
manager.update_ops()

3


In [ ]:
t_start = time.time()
best_perfs = []
for i in range(3):
    perf = manager.train()
    best_perfs.append(perf)
    manager.update_ops()
    manager.head_num*=2
    manager.epochs*=2
    manager.build_model()
    manager.model.print_ops()
print(best_perfs)
print("Tot Time:",show_time(time.time()-t_start))

0:    7.83GiB
1:    9.53GiB
Epoch 00001 | Loss (L: 156.4433, C: 0.1397) | Time(s) 40.1638 | comp 0.23 | acc 0.3333 | val_acc 0.4120 | test_acc 0.4490
Epoch 00002 | Loss (L: 1.2131, C: 0.0339) | Time(s) 41.3824 | comp 0.23 | acc 0.4167 | val_acc 0.4600 | test_acc 0.4480
Epoch 00003 | Loss (L: 1.3203, C: 0.0350) | Time(s) 41.8327 | comp 0.23 | acc 0.4500 | val_acc 0.4800 | test_acc 0.4520
Epoch 00004 | Loss (L: 1.2323, C: 0.0350) | Time(s) 42.2900 | comp 0.23 | acc 0.4833 | val_acc 0.4920 | test_acc 0.4600
Epoch 00005 | Loss (L: 1.1430, C: 0.0350) | Time(s) 42.6866 | comp 0.23 | acc 0.5000 | val_acc 0.4740 | test_acc 0.4530
Epoch 00006 | Loss (L: 1.3011, C: 0.0350) | Time(s) 43.0039 | comp 0.24 | acc 0.5167 | val_acc 0.4220 | test_acc 0.4250
Epoch 00007 | Loss (L: 1.7042, C: 0.0358) | Time(s) 43.2630 | comp 0.24 | acc 0.6667 | val_acc 0.4620 | test_acc 0.4660
Deadheaded 0 ops. Remaining: 168
0: ['sum_gat_linear', 'mean_gat_linear', 'max_gat_linear', 'mlp_gat_linear', 'sum_cos_linear', 'm

In [ ]:
op_counts = [len([layer['op'].name for layer in multilayer.layers if not layer['pruner'].deadheaded]) for multilayer in manager.model.layers]

In [ ]:
def get_random_ops(n_ops):
    layers_ops = []
    for layer in n_ops:
        layer_ops = []
        for i in range(layer):
            att = random.choice(list(attents.keys()))
            agg = random.choice(list(aggs.keys()))
            act = random.choice(list(acts.keys()))
            layer_ops.append({'att':att,'agg':agg,'act':act})
        layers_ops.append(layer_ops)
    return layers_ops

In [ ]:
rand_perfs = []
for i in range(50):
    kwargs = {
        'dataset':'cora',
        'head_num':64,
        'num_layers':2,
        'prune':False,
        'epochs':128,
        'layers_ops':get_random_ops(op_counts)
    }
    print("0: "+"\n1: ".join([", ".join(["_".join(x.values()) for x in layer]) for layer in kwargs['layers_ops']]))
    manager = CitationGNNManager(kwargs)
    perf = manager.train()
    rand_perfs.append(perf)
print(max(rand_perfs),np.mean(rand_perfs))